In [7]:
from sklearn import datasets
from sklearn.model_selection import train_test_split
import numpy as np
import tensorflow as tf
from sklearn.preprocessing import OrdinalEncoder

In [8]:
X = datasets.fetch_covtype().data[:5000]
y = datasets.fetch_covtype().target[:5000]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1)

print(X_train.shape, y_train.shape)

enc = OrdinalEncoder()
y_train = enc.fit_transform(y_train.reshape(-1, 1)).reshape(-1, )
y_test = enc.transform(y_test.reshape(-1, 1)).reshape(-1, )
print(np.unique(y_train))  # 7分类任务

X_train, X_test, y_train, y_test = tf.constant(X_train), tf.constant(X_test), tf.constant(y_train), tf.constant(y_test)

(4500, 54) (4500,)
[0. 1. 2. 3. 4. 5. 6.]


In [9]:
inputs = tf.keras.Input(shape=(54,))
x = tf.keras.layers.Dense(256, activation='tanh')(inputs)
x = tf.keras.layers.Dropout(0.2)(x)
x = tf.keras.layers.Dense(128, activation='relu')(x)
x = tf.keras.layers.Dense(128)(x)
predictions = tf.keras.layers.Dense(7)(x)
model = tf.keras.Model(inputs=inputs, outputs=predictions)

model.summary()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 54)]              0         
                                                                 
 dense_4 (Dense)             (None, 256)               14080     
                                                                 
 dropout_1 (Dropout)         (None, 256)               0         
                                                                 
 dense_5 (Dense)             (None, 128)               32896     
                                                                 
 dense_6 (Dense)             (None, 128)               16512     
                                                                 
 dense_7 (Dense)             (None, 7)                 903       
                                                                 
Total params: 64,391
Trainable params: 64,391
Non-trainable

In [10]:
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True), metrics=['accuracy'])

In [11]:
sample_weight = np.ones(shape=(len(y_train),))

# 类别为5的样本权重设置为2
# 通常用于不平衡的分类问题
sample_weight[y_train == 5] = 2.
sample_weight = tf.constant(sample_weight)
sample_weight

<tf.Tensor: shape=(4500,), dtype=float64, numpy=array([2., 2., 1., ..., 1., 2., 1.])>

In [12]:
# Optional Numpy array of weights for the training samples, used for weighting the loss function (during training only).
model.fit(X_train, y_train, epochs=10, validation_data=(X_test, y_test), sample_weight=sample_weight)

Epoch 1/10
141/141 [==============================] - 1s 4ms/step - loss: 1.5657 - accuracy: 0.4291 - val_loss: 1.2634 - val_accuracy: 0.4800
Epoch 2/10
141/141 [==============================] - 1s 4ms/step - loss: 1.3969 - accuracy: 0.4756 - val_loss: 1.1672 - val_accuracy: 0.5240